# Modules and Classes

Path setup for libs:

In [1]:
import sys,os
sys.path.append(os.path.expanduser('./libitmal'))

from libitmal import utils as itmalutils
print(dir(itmalutils))
print(itmalutils.__file__)

['AssertInRange', 'CheckFloat', 'InRange', 'Iterable', 'PrintMatrix', 'ResetRandom', 'TEST', 'TestAll', 'TestCheckFloat', 'TestPrintMatrix', 'TestVarName', 'VarName', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'ctxlib', 'inf', 'inspect', 'isFloat', 'isList', 'isNumpyArray', 'nan', 'np', 'random', 're']
/Users/esbeninglev/Dropbox/Documents/Uni/6_semester/MAL_MachineLearning/mal_grp30/O1/libitmal/utils.py


### Qa - Load and test libitmal

In [2]:
from libitmal import utils as itmalutils

itmalutils.TestAll()

TestPrintMatrix...(no regression testing)
X=[[   1.    2.]
   [   3. -100.]
   [   1.   -1.]]
X=[[ 1.  2.]
   ...
   [ 1. -1.]]
X=[[   1.
       2.    ]
   [   3.0001
    -100.    ]
   [   1.
      -1.    ]]
X=[[   1.    2.]
   [   3. -100.]
   [   1.   -1.]]
OK
TEST: OK
ALL OK


### Qb Create your own module, with some functions, and test it

In [4]:
import malutils

malutils.HelloWorld()
malutils.Greeter("Pokemon!")

Hello World!
Hello Pokemon!!


### Qc How do you 'recompile' a module?

#### Answer
Reload of modules can be done in serveral ways. One simple way is to just restart the kernal.
Another is the code below.

In [5]:
import importlib
importlib.reload(malutils)


<module 'malutils' from '/Users/esbeninglev/Dropbox/Documents/Uni/6_semester/MAL_MachineLearning/mal_grp30/O1/malutils/__init__.py'>

# Intro

### Qa) The $\theta$ parameters and the $R^2$ Score


### Qb) Using k-Nearest Neighbors


### Qc) Tuning Parameter for k-Nearest Neighbors and A Sanity Check


### Qd) Trying out a Neural Network
